# Webscrapping the images

Last analysis I wanted to do was to process the covers of each movie and TV show using computer vision.

In this notebook, I scrapped the images from FilmAffinity and prepared the data to process it later in the notebook called *Computer Vision Analysis: color image processing*.

To finish this part, I tried to apply some face recognition to detect the gender of each cover in *Computer Vision Analysis: gender image detection*.

In [32]:
import pandas as pd

from bs4 import BeautifulSoup

import requests
import urllib.request
import shutil

## Reading the dataset

To start this process I read the .csv file obtained from the EDA.

In [33]:
data = pd.read_csv('datasets/final_dataset_clean.csv',index_col=0)
data.head()

,Title,Link,Type,Year_Release,Country,Directors,Women_Directors,Men_Directors,Not_Set_Directors,Total_Directors,...,%_Not_Set_Cast,Genres,Description,RatingAverage,Votes,Reviews,Country_Code,Continent,Most_Common_Gender_Cast,Most_Common_Gender_Directors
0,Money Heist,https://www.filmaffinity.com/us/film879405.html,TV show,2017,Spain,"Álex Pina, Jesús Colmenar, Miguel Ángel Vivas,...",0,5,2,7,...,5.13,"TV Series, Thriller, Mystery, Heist Film, Kidn...",TV Series (2017-Present Day). 4 Seasons. A mys...,7.1,25691.0,"""[4th Season Review]: [It] is like an extended...",ESP,Europe,Men,Men
1,The Blacklist,https://www.filmaffinity.com/us/film573633.html,TV show,2013,United States of America,"Jon Bokenkamp, Michael W. Watkins, Andrew McCa...",4,32,0,36,...,0.00,"TV Series, Mystery, Drama, Crime, Spy Film","The world's most wanted criminal, Thomas Raymo...",6.4,5148.0,"""His name is above the title and, depending ho...",USA,Americas,Men,Men
2,Locked Up,https://www.filmaffinity.com/us/film441483.html,TV show,2015,Spain,"Iván Escobar, Esther Martínez Lobato, Daniel É...",2,3,4,9,...,6.25,"TV Series, Thriller, Drama, Prison Drama",Macarena Ferreiro is a young naive woman who f...,7.0,6941.0,NaN,ESP,Europe,Women,Men
3,Prison Break,https://www.filmaffinity.com/us/film822756.html,TV show,2005,United States of America,"Paul Scheuring, Bobby Roth, Kevin Hooks, Dwigh...",1,30,1,32,...,9.52,"TV Series, Action, Drama, Prison Drama, Cop Mo...",TV Series (2005-2009). 5 Seasons. 90 Episodes....,7.3,71511.0,"""A strong cast led by Wentworth Miller (...) I...",USA,Americas,Men,Men
4,13 Reasons Why,https://www.filmaffinity.com/us/film999360.html,TV show,2017,United States of America,"Brian Yorkey, Tom McCarthy, Kyle Patrick Alvar...",2,5,0,7,...,2.86,"TV Series, Drama, Mystery, Teen/coming-of-age,...","'Thirteen Reasons Why', based on the best-sell...",6.8,21496.0,"""[2nd Season Review]: [It] is a frustratingly ...",USA,Americas,Men,Men


In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2186 entries, 0 to 2185
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Title                         2186 non-null   object 
 1   Link                          2186 non-null   object 
 2   Type                          2186 non-null   object 
 3   Year_Release                  2186 non-null   int64  
 4   Country                       2186 non-null   object 
 5   Directors                     2167 non-null   object 
 6   Women_Directors               2186 non-null   int64  
 7   Men_Directors                 2186 non-null   int64  
 8   Not_Set_Directors             2186 non-null   int64  
 9   Total_Directors               2186 non-null   int64  
 10  %_Women_Directors             2186 non-null   float64
 11  %_Men_Directors               2186 non-null   float64
 12  %_Not_Set_Directors           2186 non-null   float64
 13  Cas

## Scrapping the covers

From here, I started making requests to FilmAffinity downloading all the covers.

In [5]:
url = data.Link[0]
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

In [6]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<title>Money Heist (TV Series) (2017) - FilmAffinity</title>
<meta charset="utf-8"/>
<meta content="Money Heist (TV Series) is a TV Series directed by Álex Pina (Creator), Jesús Colmenar ... with Álvaro Morte, Úrsula Corberó, Itziar Ituño, Alba Flores .... Year: 2017. Original title: La casa de papel (TV Series). Synopsis: TV Series (2017-Present Day). 4 Seasons. A mysterious character called the Professor is planning the biggest robbery in history. To do this, he recruits eight thieves who take hostages and lock ..." name="description"/>
<meta content="FilmAffinity" property="og:site_name"/>
<meta content="summary" name="twitter:card"/>
<meta content="video.tv_show" property="og:type"/>
<meta name="twitter:site" value="@Filmaffinity"/>
<meta content="https://www.filmaffinity.com/us/film879405.html" property="og:url">
<meta content="Money Heist (TV Series) (2017)" property="og:title">
<meta content="Director: Álex Pina (Creator), Jesús Colmenar 

In [8]:
img = soup.find("img", itemprop='image')
img

<img alt="Money Heist (TV Series)" height="246" itemprop="image" src="https://pics.filmaffinity.com/la_casa_de_papel_tv_series-307600464-mmed.jpg" width="160"/>

In [10]:
image_info = []
image_info.append((img["src"], img["alt"]))
image_info

[('https://pics.filmaffinity.com/la_casa_de_papel_tv_series-307600464-mmed.jpg',
  'Money Heist (TV Series)')]

In [35]:
def download_image(image):
    print(image[0])
    response = requests.get(image[0], stream=True)
    print(response)
    realname = ''.join(e for e in image[1] if e.isalnum())
    
    file = open("/Users/gina/Documents/allWomen/Final project/covers//{}.jpg".format(realname), 'wb')
    
    response.raw.decode_content = True
    shutil.copyfileobj(response.raw, file)
    print(response.raw)
    print(response.raw == None, '\n')
    del response
    
    img_path = 'covers/{}.jpg'.format(realname)
    
    return img_path

In [43]:
download_image(image_info[0])

https://pics.filmaffinity.com/la_casa_de_papel_tv_series-307600464-mmed.jpg
<Response [200]>
False 



In [36]:
image_info = []

for link in data.Link:
    print(link)
    film = requests.get(link)
    soupFilm = BeautifulSoup(film.text, 'html.parser')
    
    # image cover
    img = soupFilm.find("img", itemprop='image')
    print(img)
    print(img == None, '\n')
    
    if (img) != None:
        image_info.append((img["src"], img["alt"]))
    else:
        image_info.append('NaN')

https://www.filmaffinity.com/us/film879405.html
<img alt="Money Heist (TV Series)" height="246" itemprop="image" src="https://pics.filmaffinity.com/la_casa_de_papel_tv_series-307600464-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film573633.html
<img alt="The Blacklist (TV Series)" height="214" itemprop="image" src="https://pics.filmaffinity.com/the_blacklist_tv_series-396285612-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film441483.html
<img alt="Locked Up (TV Series)" height="224" itemprop="image" src="https://pics.filmaffinity.com/vis_a_vis_tv_series-704930886-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film822756.html
<img alt="Prison Break (TV Series)" height="229" itemprop="image" src="https://pics.filmaffinity.com/prison_break_tv_series-793100624-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film999360.html
<img alt="Thirteen Reasons Why (TV Series)" height="238" itemprop="image" src="https://pics.filma

<img alt="The Mafia Dolls (TV Series)" height="245" itemprop="image" src="https://pics.filmaffinity.com/las_munecas_de_la_mafia_2_tv_series-201566595-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film120318.html
<img alt="Iron Fist (TV Series)" height="238" itemprop="image" src="https://pics.filmaffinity.com/iron_fist_tv_series-812373264-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film632552.html
<img alt="Fauda (TV Series)" height="225" itemprop="image" src="https://pics.filmaffinity.com/fauda_tv_series-910988967-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film203401.html
<img alt="Better Call Saul (TV Series)" height="238" itemprop="image" src="https://pics.filmaffinity.com/better_call_saul_tv_series-317222694-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film678702.html
<img alt="Mindhunter (TV Series)" height="237" itemprop="image" src="https://pics.filmaffinity.com/mindhunter_tv_series-717879416-mmed.jpg" 

<img alt="Teen Titans Go! (TV Series)" height="246" itemprop="image" src="https://pics.filmaffinity.com/teen_titans_go_tv_series-351970904-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film699539.html
<img alt="Chica Vampiro (TV Series)" height="241" itemprop="image" src="https://pics.filmaffinity.com/chica_vampiro_tv_series-762841128-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film120015.html
<img alt="Fuller House (TV Series)" height="238" itemprop="image" src="https://pics.filmaffinity.com/fuller_house_tv_series-106548936-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film112263.html
<img alt="The Boss Baby: Back in Business (TV Series)" height="236" itemprop="image" src="https://pics.filmaffinity.com/the_boss_baby_back_in_business_tv_series-315390453-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film322094.html
<img alt="Never Have I Ever (TV Series)" height="234" itemprop="image" src="https://pics.filmaffinit

<img alt="Rabbids Invasion (TV Series)" height="240" itemprop="image" src="https://pics.filmaffinity.com/rabbids_invasion_tv_series-741310814-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film823181.html
<img alt="We Bare Bears (TV Series)" height="210" itemprop="image" src="https://pics.filmaffinity.com/we_bare_bears_tv_series-645005896-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film793527.html
<img alt="Grace and Frankie (TV Series)" height="238" itemprop="image" src="https://pics.filmaffinity.com/grace_and_frankie_tv_series-101831163-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film575130.html
<img alt="On My Block (TV Series)" height="238" itemprop="image" src="https://pics.filmaffinity.com/on_my_block_tv_series-513512438-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film138199.html
<img alt="I Am Not Okay with This (TV Series)" height="238" itemprop="image" src="https://pics.filmaffinity.com/i_am_not_okay_

<img alt="Pokémon the Series: Sun &amp; Moon—Ultra Adventures (TV Series)" height="225" itemprop="image" src="https://pics.filmaffinity.com/pokemon_the_series_sun_moon_ultra_adventures_tv_series-855616154-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film499162.html
<img alt="Minecraft: Story Mode (TV Series)" height="230" itemprop="image" src="https://pics.filmaffinity.com/minecraft_story_mode_tv_series-698986812-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film109990.html
<img alt="Mako Mermaids (TV Series)" height="228" itemprop="image" src="https://pics.filmaffinity.com/mako_mermaids_tv_series-797158076-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film723586.html
<img alt="The Seven Deadly Sins (TV Series)" height="228" itemprop="image" src="https://pics.filmaffinity.com/nanatsu_no_taizai_tv_series-155449423-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film920440.html
<img alt="The Crown (TV Series)" height=

...

<img alt="Inside Edge (Serie de TV)" height="238" itemprop="image" src="https://pics.filmaffinity.com/inside_edge_tv_series-227537726-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film539862.html
<img alt="Breathe (Serie de TV)" height="238" itemprop="image" src="https://pics.filmaffinity.com/breathe_tv_series-337292423-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film426026.html
<img alt="Loudermilk (Serie de TV)" height="216" itemprop="image" src="https://pics.filmaffinity.com/loudermilk_tv_series-895633056-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film585818.html
<img alt="Patriota (Serie de TV)" height="238" itemprop="image" src="https://pics.filmaffinity.com/patriot_tv_series-392558401-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film538230.html
<img alt="The Bold Type (Serie de TV)" height="238" itemprop="image" src="https://pics.filmaffinity.com/the_bold_type_tv_series-507921793-mmed.jpg" width="160"/>

<img alt="El sexo y la Sra. X (TV)" height="221" itemprop="image" src="https://pics.filmaffinity.com/sex_mrs_x_tv-285812922-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film910887.html
<img alt="Vivir y otras ficciones " height="227" itemprop="image" src="https://pics.filmaffinity.com/vivir_y_otras_ficciones-864908670-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film231543.html
<img alt="Bronce " height="227" itemprop="image" src="https://pics.filmaffinity.com/bronce-572424763-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film540918.html
<img alt="Grand Blue (Serie de TV)" height="231" itemprop="image" src="https://pics.filmaffinity.com/grand_blue_tv_series-444180804-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film376818.html
<img alt="La maravillosa Sra. Maisel (Serie de TV)" height="240" itemprop="image" src="https://pics.filmaffinity.com/the_marvelous_mrs_maisel_tv_series-381789063-mmed.jpg" width="160"/>
Fa

<img alt="Forky hace una pregunta: ¿Qué es el tiempo? (Ep) (C)" height="240" itemprop="image" src="https://pics.filmaffinity.com/forky_asks_a_question_what_is_time_tv_s-295518689-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film827774.html
<img alt="Eduardo Manostijeras " height="244" itemprop="image" src="https://pics.filmaffinity.com/edward_scissorhands-367797250-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film184816.html
<img alt="Carlitos y Snoopy: La película de Peanuts " height="237" itemprop="image" src="https://pics.filmaffinity.com/snoopy_and_charlie_brown_the_peanuts_movie-683129408-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film653715.html
<img alt="Los elefantes " height="229" itemprop="image" src="https://pics.filmaffinity.com/elephant-876176255-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film608278.html
<img alt="Delfines, la vida en el arrecife " height="235" itemprop="image" src="https://pic

<img alt="El rayo mágico (C)" height="231" itemprop="image" src="https://pics.filmaffinity.com/short_circuit_lightning_in_a_bottle_s-811210904-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film644211.html
<img alt="La carrera (C)" height="231" itemprop="image" src="https://pics.filmaffinity.com/short_circuit_the_race_s-915534714-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film294683.html
<img alt="Pel-Jitsu (C)" height="229" itemprop="image" src="https://pics.filmaffinity.com/short_circuit_hair_jitsu_s-674417691-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film103261.html
<img alt="Downtown (C)" height="240" itemprop="image" src="https://pics.filmaffinity.com/short_circuit_downtown_s-428285671-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film508974.html
<img alt="Jing Hua (Flor en el espejo) (C)" height="233" itemprop="image" src="https://pics.filmaffinity.com/short_circuit_jing_hua_s-892084657-mmed.jpg" width=

<img alt="Visibilidad: LGTBI en la televisión (Serie de TV)" height="225" itemprop="image" src="https://pics.filmaffinity.com/visible_out_on_television_tv_series-478993057-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film239610.html
<img alt="Cuentos asombrosos (Serie de TV)" height="240" itemprop="image" src="https://pics.filmaffinity.com/amazing_stories_tv_series-207391582-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film671786.html
<img alt="Beastie Boys Story " height="240" itemprop="image" src="https://pics.filmaffinity.com/beastie_boys_story-247954972-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film707527.html
<img alt="Defender a Jacob (Miniserie de TV)" height="240" itemprop="image" src="https://pics.filmaffinity.com/defending_jacob-740018610-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film734275.html
<img alt="Ciclos (Serie de TV)" height="238" itemprop="image" src="https://pics.filmaffinity.com/tryi

In [37]:
len(image_info)

2186

In [38]:
data['Cover_Image_Info'] = image_info
data.head()

,Title,Link,Type,Year_Release,Country,Directors,Women_Directors,Men_Directors,Not_Set_Directors,Total_Directors,...,Genres,Description,RatingAverage,Votes,Reviews,Country_Code,Continent,Most_Common_Gender_Cast,Most_Common_Gender_Directors,Cover_Image_Info
0,Money Heist,https://www.filmaffinity.com/us/film879405.html,TV show,2017,Spain,"Álex Pina, Jesús Colmenar, Miguel Ángel Vivas,...",0,5,2,7,...,"TV Series, Thriller, Mystery, Heist Film, Kidn...",TV Series (2017-Present Day). 4 Seasons. A mys...,7.1,25691.0,"""[4th Season Review]: [It] is like an extended...",ESP,Europe,Men,Men,(https://pics.filmaffinity.com/la_casa_de_pape...
1,The Blacklist,https://www.filmaffinity.com/us/film573633.html,TV show,2013,United States of America,"Jon Bokenkamp, Michael W. Watkins, Andrew McCa...",4,32,0,36,...,"TV Series, Mystery, Drama, Crime, Spy Film","The world's most wanted criminal, Thomas Raymo...",6.4,5148.0,"""His name is above the title and, depending ho...",USA,Americas,Men,Men,(https://pics.filmaffinity.com/the_blacklist_t...
2,Locked Up,https://www.filmaffinity.com/us/film441483.html,TV show,2015,Spain,"Iván Escobar, Esther Martínez Lobato, Daniel É...",2,3,4,9,...,"TV Series, Thriller, Drama, Prison Drama",Macarena Ferreiro is a young naive woman who f...,7.0,6941.0,NaN,ESP,Europe,Women,Men,(https://pics.filmaffinity.com/vis_a_vis_tv_se...
3,Prison Break,https://www.filmaffinity.com/us/film822756.html,TV show,2005,United States of America,"Paul Scheuring, Bobby Roth, Kevin Hooks, Dwigh...",1,30,1,32,...,"TV Series, Action, Drama, Prison Drama, Cop Mo...",TV Series (2005-2009). 5 Seasons. 90 Episodes....,7.3,71511.0,"""A strong cast led by Wentworth Miller (...) I...",USA,Americas,Men,Men,(https://pics.filmaffinity.com/prison_break_tv...
4,13 Reasons Why,https://www.filmaffinity.com/us/film999360.html,TV show,2017,United States of America,"Brian Yorkey, Tom McCarthy, Kyle Patrick Alvar...",2,5,0,7,...,"TV Series, Drama, Mystery, Teen/coming-of-age,...","'Thirteen Reasons Why', based on the best-sell...",6.8,21496.0,"""[2nd Season Review]: [It] is a frustratingly ...",USA,Americas,Men,Men,(https://pics.filmaffinity.com/13_reasons_why_...


In [39]:
# Repeat the process in order to get the information from NaN values
for item in data.index:
    print(data.Link[item])
    if (data.Cover_Image_Info[item]) == 'NaN':
        film = requests.get(data.Link[item])
        soupFilm = BeautifulSoup(film.text, 'html.parser')
        
        # image cover
        img = soupFilm.find("img", itemprop='image')
        print(img)
        print(img == None, '\n')
        
        if (img) != None:
            data.Cover_Image_Info[item] = (img["src"], img["alt"])
        else:
            data.Cover_Image_Info[item] = 'NaN'

https://www.filmaffinity.com/us/film879405.html
https://www.filmaffinity.com/us/film573633.html
https://www.filmaffinity.com/us/film441483.html
https://www.filmaffinity.com/us/film822756.html
https://www.filmaffinity.com/us/film999360.html
https://www.filmaffinity.com/us/film103430.html
https://www.filmaffinity.com/us/film910593.html
https://www.filmaffinity.com/us/film482516.html
https://www.filmaffinity.com/us/film539830.html
https://www.filmaffinity.com/us/film340588.html
https://www.filmaffinity.com/us/film512020.html
https://www.filmaffinity.com/us/film654725.html
https://www.filmaffinity.com/us/film187456.html
https://www.filmaffinity.com/us/film489970.html
https://www.filmaffinity.com/us/film966117.html
https://www.filmaffinity.com/us/film718452.html
https://www.filmaffinity.com/us/film194842.html
https://www.filmaffinity.com/us/film880530.html
https://www.filmaffinity.com/us/film828822.html
https://www.filmaffinity.com/us/film668282.html
https://www.filmaffinity.com/us/film9402

<img alt="Los dos papas " height="237" itemprop="image" src="https://pics.filmaffinity.com/the_two_popes-532690428-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/au/film557017.html
<img alt="Falling Inn Love " height="238" itemprop="image" src="https://pics.filmaffinity.com/falling_inn_love-427238856-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film594904.html


/opt/anaconda3/envs/GinaAllWomen/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


<img alt="La cita perfecta " height="238" itemprop="image" src="https://pics.filmaffinity.com/the_perfect_date-939685044-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film155713.html
<img alt="Grand-Daddy Day Care " height="226" itemprop="image" src="https://pics.filmaffinity.com/grand_daddy_day_care-224771597-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film427424.html
<img alt="The Platform " height="229" itemprop="image" src="https://pics.filmaffinity.com/el_hoyo-987825598-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film994025.html
<img alt="The Invisible Guest " height="228" itemprop="image" src="https://pics.filmaffinity.com/contratiempo-621094614-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film930503.html
<img alt="Old Boy " height="231" itemprop="image" src="https://pics.filmaffinity.com/oldboy_old_boy-260793961-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/us/film143772.html
<img alt="Th

https://www.filmaffinity.com/es/film819887.html
https://www.filmaffinity.com/es/film562470.html
https://www.filmaffinity.com/es/film857434.html
https://www.filmaffinity.com/es/film594689.html
https://www.filmaffinity.com/es/film793070.html
https://www.filmaffinity.com/es/film483742.html
https://www.filmaffinity.com/es/film967439.html
https://www.filmaffinity.com/es/film670626.html
https://www.filmaffinity.com/es/film367689.html
https://www.filmaffinity.com/es/film954631.html
https://www.filmaffinity.com/es/film285036.html
https://www.filmaffinity.com/es/film636712.html
https://www.filmaffinity.com/es/film508114.html
https://www.filmaffinity.com/es/film124619.html
https://www.filmaffinity.com/es/film663671.html
https://www.filmaffinity.com/es/film254051.html
https://www.filmaffinity.com/es/film606301.html
https://www.filmaffinity.com/es/film673299.html
https://www.filmaffinity.com/es/film852380.html
https://www.filmaffinity.com/es/film497000.html
https://www.filmaffinity.com/es/film2857

<img alt="La querida del Centauro (Serie de TV)" height="235" itemprop="image" src="https://pics.filmaffinity.com/la_querida_del_centauro_tv_series-382987573-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film804155.html
<img alt="Yo, mi mujer y mi mujer muerta " height="226" itemprop="image" src="https://pics.filmaffinity.com/yo_mi_mujer_y_mi_mujer_muerta-517935291-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film843451.html
<img alt="El emperador de París " height="218" itemprop="image" src="https://pics.filmaffinity.com/l_empereur_de_paris-110091678-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film978970.html
<img alt="Spider-Man: Lejos de casa " height="238" itemprop="image" src="https://pics.filmaffinity.com/spider_man_far_from_home-339542528-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film178719.html
<img alt="Virtual Hero: La Serie (Serie de TV)" height="221" itemprop="image" src="https://pics.filmaffinit

<img alt="El cuento de las comadrejas " height="226" itemprop="image" src="https://pics.filmaffinity.com/el_cuento_de_las_comadrejas-237163843-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film358100.html
<img alt="Yesterday " height="238" itemprop="image" src="https://pics.filmaffinity.com/yesterday-541518094-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film908082.html
<img alt="La banda " height="228" itemprop="image" src="https://pics.filmaffinity.com/la_banda-869944417-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film476901.html
<img alt="Clara y Claire " height="218" itemprop="image" src="https://pics.filmaffinity.com/celle_que_vous_croyez-492727105-mmed.jpg" width="160"/>
False 

https://www.filmaffinity.com/es/film997700.html
<img alt="Largo viaje hacia la noche " height="241" itemprop="image" src="https://pics.filmaffinity.com/di_qiu_zui_hou_de_ye_wan_long_day_s_journey_into_night-821907675-mmed.jpg" width="161"/>
False 



In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2186 entries, 0 to 2185
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Title                         2186 non-null   object 
 1   Link                          2186 non-null   object 
 2   Type                          2186 non-null   object 
 3   Year_Release                  2186 non-null   int64  
 4   Country                       2186 non-null   object 
 5   Directors                     2167 non-null   object 
 6   Women_Directors               2186 non-null   int64  
 7   Men_Directors                 2186 non-null   int64  
 8   Not_Set_Directors             2186 non-null   int64  
 9   Total_Directors               2186 non-null   int64  
 10  %_Women_Directors             2186 non-null   float64
 11  %_Men_Directors               2186 non-null   float64
 12  %_Not_Set_Directors           2186 non-null   float64
 13  Cas

In [43]:
images_path = []

for i in data.Cover_Image_Info:
    images_path.append(download_image(i))

https://pics.filmaffinity.com/la_casa_de_papel_tv_series-307600464-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/the_blacklist_tv_series-396285612-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/vis_a_vis_tv_series-704930886-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/prison_break_tv_series-793100624-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/13_reasons_why_tv_series-797870985-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/dark_tv_series-111203947-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/riverdale_tv_series-279637679-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/lucifer_tv_series-145111876-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/peaky_blinders_tv_series-713495787-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/la_reina_del_flow_tv_series-223343429-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/jeffr

<Response [200]>
False 

https://pics.filmaffinity.com/kara_para_ask_tv_series-909738488-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/la_bella_y_las_bestias_tv_series-378995509-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/el_recluso_tv_series-158001494-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/daredevil_tv_series-563047573-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/alta_mar_tv_series-727717289-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/falsa_identidad_tv_series-513729769-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/the_disappearance_of_madeleine_mccann-904171151-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/the_alienist-840831653-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/dare_me_tv_series-730719713-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/broadchurch_tv_series-909016734-mmed.jpg
<Response [200]>
False 

h

<Response [200]>
False 

https://pics.filmaffinity.com/los_briceno_tv_series-273339232-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/regular_show_tv_series-654107624-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/glee_tv_series-799183052-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/the_inbestigators_tv_series-759758946-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/the_big_show_show_tv_series-500994721-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/workin_moms_tv_series-697365744-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/the_good_place_tv_series-605671019-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/rabbids_invasion_tv_series-741310814-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/we_bare_bears_tv_series-645005896-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/grace_and_frankie_tv_series-101831163-mmed.jpg
<Response [200]>
F

<Response [200]>
False 

https://pics.filmaffinity.com/free_rein_tv_series-270622717-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/teen_wolf_tv_series-614017053-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/the_last_kingdom_tv_series-305344254-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/barbie_dreamhouse_adventures_tv_series-752388882-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/simon_tv_series-761298411-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/avatar_the_last_airbender_avatar_the_legend_of_aang_tv_series-118276707-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/the_witcher_tv_series-330642357-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/gossip_girl_tv_series-386731829-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/unorthodox-981675828-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/masha_i_medved_masha_and_the_bear_tv_ser

<Response [200]>
False 

https://pics.filmaffinity.com/boku_dake_ga_inai_machi_tv_series-157306262-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/saiko_pasu_psycho_pass_tv_series-973639582-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/no_game_no_life_tv_series-176375261-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/sainto_seiya_za_rosuto_kyanbasu_meio_sinwa_saint_seiya_the_lost_canvas_hades_mythology_tv_series-718086768-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/saiki_kusuo_no_sai_nan_tv_series-515884634-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/smile_precure_tv_series-993018231-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/riraifu_relife_tv_series-130728077-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/kodo_giasu_hangyaku_no_rurushu_code_geass_lelouch_of_the_rebellion_tv_series-254439377-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/devi

...

<Response [200]>
False 

https://pics.filmaffinity.com/the_great_buster-619271373-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/premieres_vacances-677762649-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/ayla-456620354-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/teen_spirit-715237820-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/the_song_of_sway_lake-273671507-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/l_empire_de_la_perfection-416014312-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/ti_presento_sofia-932247295-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/feng_huo_fang_fei-807517511-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/auf_der_suche_nach_ingmar_bergman-285532423-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/the_eyes_of_orson_welles-552363243-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/varda_by_

<Response [200]>
False 

https://pics.filmaffinity.com/luis_el_sabio_del_exito-117272208-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/pequenas_coincidencias_tv_series-260167225-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/molly_s_game-652441333-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/six_dreams_tv_series-819835264-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/gringo-730732457-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/the_dangerous_book_for_boys_tv_series-416159911-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/same_kind_of_different_as_me-606139443-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/the_last_witch-246842123-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/the_looming_tower-889932134-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/mcmafia_tv_series-923365952-mmed.jpg
<Response [200]>
False 

https://pics.filmaf

<Response [200]>
False 

https://pics.filmaffinity.com/in_april-105049496-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/jack_ryan_tv_series-348234630-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/creative_galaxy_tv_series-577590838-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/transparent_tv_series-404082785-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/z_the_beginning_of_everything_tv_series-436531219-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/the_kicks_tv_series-614993066-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/one_mississippi_tv_series-316340319-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/you_are_wanted_tv_series-190924581-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/mariposas_verdes-559044103-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/codumentary-414026119-mmed.jpg
<Response [200]>
False 

https://pics.filma

<Response [200]>
False 

https://pics.filmaffinity.com/snoopy_and_charlie_brown_the_peanuts_movie-683129408-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/elephant-876176255-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/dolphin_reef-243212681-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/forky_asks_a_question_what_is_art_tv_s-416544890-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/penguins-584331832-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/penguins_life_on_the_edge-884235777-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/the_mandalorian_tv_series-526462730-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/in_the_footsteps_of_elephant-884507278-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/forky_asks_a_question_what_is_a_friend_tv_s-261907476-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/diving_with_dolphins-674296597-mmed.jpg
<

<Response [200]>
False 

https://pics.filmaffinity.com/amazing_stories_dynoman_and_the_volt_tv-836901732-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/amazing_stories_the_heat_tv-135917816-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/amazing_stories_the_cellar_tv-811574778-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/see_tv_series-441964718-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/truth_be_told_tv_series-211579974-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/servant_tv_series-412164128-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/the_banker-297565830-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/dads-988985061-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/the_morning_show_tv_series-346386085-mmed.jpg
<Response [200]>
False 

https://pics.filmaffinity.com/here_we_are_notes_for_living_on_planet_earth-394883016-mmed.jpg
<Response [200]>
F

In [44]:
len(images_path)

2186

In [45]:
data['Cover_Image_Path'] = images_path
data.head()

,Title,Link,Type,Year_Release,Country,Directors,Women_Directors,Men_Directors,Not_Set_Directors,Total_Directors,...,Description,RatingAverage,Votes,Reviews,Country_Code,Continent,Most_Common_Gender_Cast,Most_Common_Gender_Directors,Cover_Image_Info,Cover_Image_Path
0,Money Heist,https://www.filmaffinity.com/us/film879405.html,TV show,2017,Spain,"Álex Pina, Jesús Colmenar, Miguel Ángel Vivas,...",0,5,2,7,...,TV Series (2017-Present Day). 4 Seasons. A mys...,7.1,25691.0,"""[4th Season Review]: [It] is like an extended...",ESP,Europe,Men,Men,(https://pics.filmaffinity.com/la_casa_de_pape...,covers/MoneyHeistTVSeries.jpg
1,The Blacklist,https://www.filmaffinity.com/us/film573633.html,TV show,2013,United States of America,"Jon Bokenkamp, Michael W. Watkins, Andrew McCa...",4,32,0,36,...,"The world's most wanted criminal, Thomas Raymo...",6.4,5148.0,"""His name is above the title and, depending ho...",USA,Americas,Men,Men,(https://pics.filmaffinity.com/the_blacklist_t...,covers/TheBlacklistTVSeries.jpg
2,Locked Up,https://www.filmaffinity.com/us/film441483.html,TV show,2015,Spain,"Iván Escobar, Esther Martínez Lobato, Daniel É...",2,3,4,9,...,Macarena Ferreiro is a young naive woman who f...,7.0,6941.0,NaN,ESP,Europe,Women,Men,(https://pics.filmaffinity.com/vis_a_vis_tv_se...,covers/LockedUpTVSeries.jpg
3,Prison Break,https://www.filmaffinity.com/us/film822756.html,TV show,2005,United States of America,"Paul Scheuring, Bobby Roth, Kevin Hooks, Dwigh...",1,30,1,32,...,TV Series (2005-2009). 5 Seasons. 90 Episodes....,7.3,71511.0,"""A strong cast led by Wentworth Miller (...) I...",USA,Americas,Men,Men,(https://pics.filmaffinity.com/prison_break_tv...,covers/PrisonBreakTVSeries.jpg
4,13 Reasons Why,https://www.filmaffinity.com/us/film999360.html,TV show,2017,United States of America,"Brian Yorkey, Tom McCarthy, Kyle Patrick Alvar...",2,5,0,7,...,"'Thirteen Reasons Why', based on the best-sell...",6.8,21496.0,"""[2nd Season Review]: [It] is a frustratingly ...",USA,Americas,Men,Men,(https://pics.filmaffinity.com/13_reasons_why_...,covers/ThirteenReasonsWhyTVSeries.jpg


## Saving the data

Finally, I saved the new DataFrame into another .csv file, which contained the paths where the covers of each movie and TV show were located.

In [46]:
data.to_csv('datasets/final_dataset_clean_coverInfo.csv')